# Sentiment Analysis with LSTMs

In this notebook, we'll implement a recurrent neural network that performs sentiment analysis.

## 0. Load in and visualize the data

In [1]:
!mkdir data
!wget -c https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/labels.txt
!wget -c https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/reviews.txt
!mv *.txt data/

--2022-05-22 06:06:39--  https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/labels.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/agungsantoso/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt [following]
--2022-05-22 06:06:39--  https://raw.githubusercontent.com/agungsantoso/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.02s   

2022-05-22 06:06:40 (8.91 MB/

In [2]:
# see a sample of the dataset
import numpy as np

with open("data/reviews.txt", 'r') as f:
    reviews = f.read()

with open("data/labels.txt", 'r') as f:
    labels = f.read()
    
print(reviews[:1000])
print()
print(labels[:30])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

## 1. Data pre-processing


In [3]:
from string import punctuation

print(f"Built-in punctuations: {punctuation}")

# get rid of punctuations
reviews = reviews.lower() # lower
all_text = ''.join([c for c in reviews if c not in punctuation])

Built-in punctuations: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [4]:
# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

### Encoding the words


In [5]:
from collections import Counter

# build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: i for i, word in enumerate(vocab, 1)} 

# use the dict to tokenize each review in reviews_split
# store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [6]:
# stats about the vocabulary
print("Unique words:", len(vocab_to_int))

# print tokens in first review
print("Tokenized review: \n", reviews_ints[0])

Unique words: 74072
Tokenized review: 
 [21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]


### Encoding the labels

In [7]:
# 1=positive, 0=negative
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

encoded_labels[:10]

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

### Removing outliers

In [8]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print(f"Zero-length reviews: {review_lens[0]}")
print(f"Maximum-length reviews: {max(review_lens)}")

Zero-length reviews: 1
Maximum-length reviews: 2514


In [9]:
print("Number of reviews before removing outliers: ", len(reviews_ints))

# remove any reviews/labels with zero length from the reviews_ints list
non_zero_idx = [i for i, review in enumerate(reviews_ints) if len(review)]
reviews_ints = [reviews_ints[i] for i in non_zero_idx]
encoded_labels = np.array([encoded_labels[i] for i in non_zero_idx])

print("Number of reviews after removing outliers: ", len(reviews_ints))


Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


### Padding sequences

In [10]:
def pad_features(reviews_ints, seq_length):
    
    # getting the correct (rowsxcols) shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    
    # for each review, pad from the left of the sequence
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
        
    return features

In [11]:
# teset pad_features
seq_length = 200
features = pad_features(reviews_ints, seq_length)

print(features[:30, :10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

## 2. Dataloaders and Batching

In [12]:
# split data into training, validation, and test data
split_frac = 0.8

split_idx = int(len(features)*0.8)
train_X, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_X, test_X = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

print("shape train_X:", train_X.shape)
print("shape val_X:", val_X.shape)
print("shape test_X:", test_X.shape)
print("shape train_y:", train_y.shape)
print("shape val_y:", val_y.shape)
print("shape test_y:", test_y.shape)

shape train_X: (20000, 200)
shape val_X: (2500, 200)
shape test_X: (2500, 200)
shape train_y: (20000,)
shape val_y: (2500,)
shape test_y: (2500,)


In [22]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create tensor datasets
train_data = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_X), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))

# create data loaders
batch_size = 50
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

## 3. Sentiment Network (RNN)


In [23]:
# first check if GPU is available
train_on_gpu = torch.cuda.is_available()

print("Your program will run on GPU?", train_on_gpu)

Your program will run on GPU? True


### Model

In [24]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)
        
        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if(train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                       weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                        weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [25]:
# instantiate the model
vocab_size = len(vocab_to_int) + 1 # +1 for zero padding
output_size = 1 # pos/neg
embedding_dim = 400
hidden_dim = 256 # number of units of a lstm layer
n_layers = 2 # number of lstm layers

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


## 4. Training

In [26]:
# loss and optimization functions
lr = 0.001

criterion = nn.BCELoss() # binary cross entropy loss
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [29]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model

        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.640346... Val Loss: 0.610957
Epoch: 1/4... Step: 200... Loss: 0.419593... Val Loss: 0.511323
Epoch: 1/4... Step: 300... Loss: 0.452594... Val Loss: 0.499771
Epoch: 1/4... Step: 400... Loss: 0.277503... Val Loss: 0.453776
Epoch: 2/4... Step: 500... Loss: 0.305157... Val Loss: 0.436130
Epoch: 2/4... Step: 600... Loss: 0.258760... Val Loss: 0.465048
Epoch: 2/4... Step: 700... Loss: 0.424110... Val Loss: 0.443332
Epoch: 2/4... Step: 800... Loss: 0.129274... Val Loss: 0.444443
Epoch: 3/4... Step: 900... Loss: 0.155316... Val Loss: 0.486795
Epoch: 3/4... Step: 1000... Loss: 0.254357... Val Loss: 0.476492
Epoch: 3/4... Step: 1100... Loss: 0.175948... Val Loss: 0.532475
Epoch: 3/4... Step: 1200... Loss: 0.396188... Val Loss: 0.565588
Epoch: 4/4... Step: 1300... Loss: 0.124833... Val Loss: 0.553910
Epoch: 4/4... Step: 1400... Loss: 0.156098... Val Loss: 0.564738
Epoch: 4/4... Step: 1500... Loss: 0.103322... Val Loss: 0.523830
Epoch: 4/4... Step: 1600... Loss: 

## 5. Testing

In [30]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.508
Test accuracy: 0.808
